# Setup MongoDB

# Install Mongo

    I'm going to stick with docker here. Just use the following commands to pull image and run the instance.

```python
docker pull mongo
docker run -d -p 27019:27017 -v /usr/app/mongodata:/data/db mongo
```

Just to note that we can run the mongodb in whatever the port we wish to run , however when we run the client (mongo), it'll look to bind at port 27017. Since I'm running this at the port 27019, I'll be connecting to mongodb server using the following command. Open a new terminal and connect.

mongo --port 27019


Now we'll able to connect to our mongo instance and start querying. Use the command <b> show dbs </b> and make sure you've some default dbs there. However, lets make sure we enable authentication and authorization. Use the following commands.

```python
use admin

db.createUser({user:"admin", pwd: "yourpwd", roles:["root"], passwordDigestor:"server"})
```

Here root is the super user and shouldn't be used in any of your app's connections strings. Lets create another user and user is restricted to a particular db (read and write)

```python
db.createUser({user: 'app', pwd: 'againpwd', roles:[{role:"readWrite", db:"yourdb1"}, {role:"readWrite", db:"yourdb2"}], customData:{description: 'this user is created for read write access on this db and is used in apps'},passwordDigestor:"server"})
```

Ok, let's stop the container and remove it. Now start mongo with <b>auth</b> required so that only persons who have proper credentials can query our db.

```python
docker run -d -p 27019:27017 -v /usr/app/mongodata:/data/db mongo --auth
```

# Import data

In real time scenarios we would be importing data into mongo either from relational databases or from web requests / services. It is not that uncommon having both relational and no sql databases. Most of the time the relational database will be source of truth while no sql greatly optimizes the performance of the website by making reads faster.

To transfer data from relational database(sql server) into mongo, I'll be taking out data through bcp and add into mongo using mongoimport and once imported, I'll transform the data using aggregation pipeline and move the result data into collection I need. You can also convert the data from SQL server into json and then mongoimport

And in some cases we might be taking backup and restore the data. I'll list some commands for different use cases.

```python
bcp "select * from schema.table for json path,INCLUDE_NULL_VALUES" queryout "D:\data.json" -c -S server -U usr -P pwd -d db

bcp schema.table out "/usr/app/sqldata/fiinddata.tsv" -c -S server -U 'user@server' -P pwd -d db
 
mongoimport -h host:port -d db_name -c collection_name --drop --type tsv --file "data.tsv" --fieldFile "fields.txt"

mongoimport -h host:port -d db_name -c collection_name --drop --jsonArray "fiinddata.json"

mongorestore -d db_name -c collection_name values.bson -u usr -p pwd --authenticationDatabase db_name
```

We'll be using two databases 'Companies' and 'Stocks' from the link here https://github.com/ozlerhakan/mongodb-json-files.
So download these data and load it into your mongo instance. 

We can use mongorestore when we've bson data and mongoimport when we've other data files. https://www.mongodb.com/json-and-bson

'Stocks' db schema is very simple but high in number while 'Companies' db schema is very nested. 

# Schema creation

One of the important art to learn is structuring your collection schema. When to nest a document inside another document field and when to avoid nested document and create it as separate collection. I'll give two quick examples to trigger your thoughts on this.

1. Product purchase:
    Now imagine you're using mongo db for storing records of products purchased by the customer in a store. Here, the price or discount of a product may change over time but we care only about the details of that product purchased at that time by the customer. So here nesting a document makes sense.
    
2. Book store:
    Say you're storing data about the different books. An author can have multiple books and a book might have multiple authors. Storing author details in separate collection makes more sense because we need to point the latest info about an author. Else we need to go and update in all the documents whenever an author's bio or age is changed.
    
  
Irrespective of relations (1 to 1, 1 to many, many to 1 or many to many), create the schema such that you effectively read the data from mongo. If you're ending up with aggregation pipeline most of the time, then your schema have to be changed.